In [15]:
import os
import pickle
import numpy as np
from mutagen.mp3 import MP3
from mutagen.mp4 import MP4
from mutagen.id3 import ID3, ID3NoHeaderError

max_duration = 10 * 60 # avoid adding mixes to mix

def munge(filename):
    if os.name == 'nt':
        return filename
    else:
        return filename.replace('\\', '/').replace(':', '_')
    
def get_track_duration(filename):
    duration = 0
    if filename[-3:].lower() == 'mp3':
        duration = MP3(munge(filename)).info.length
    elif filename[-3:].lower() == 'm4a':
        duration = MP4(munge(filename)).info.length
    return duration

def most_similar_by_vec(positive=[], negative=[], topn=5, noise=0):
    if isinstance(positive, str):
        positive = [positive] # broadcast to list
    if isinstance(negative, str):
        negative = [negative] # broadcast to list
    mp3_vec_i = np.sum([i for i in positive] + [-i for i in negative], axis=0)
    mp3_vec_i += np.random.normal(0, noise * np.linalg.norm(mp3_vec_i), len(mp3_vec_i))
    similar = []
    for track_j in mp3tovec:
        mp3_vec_j = mp3tovec[track_j]
        cos_proximity = np.dot(mp3_vec_i, mp3_vec_j) / (np.linalg.norm(mp3_vec_i) * np.linalg.norm(mp3_vec_j))
        similar.append((track_j, cos_proximity))
    return sorted(similar, key=lambda x:-x[1])[:topn]

def join_the_dots(tracks, n=5, noise=0): # create a musical journey between given track "waypoints"
    playlist = []
    start = tracks[0]
    start_vec = mp3tovec[start]
    for end in tracks[1:]:
        end_vec = mp3tovec[end]
        playlist.append(start)
        for i in range(n-1):
            candidates = most_similar_by_vec(positive=[(n-i+1)/n * start_vec + (i+1)/n * end_vec], topn=10, noise=noise)
            for j in range(10):
                if not candidates[j][0] in playlist \
                        and candidates[j][0] != start \
                        and candidates[j][0] != end \
                        and get_track_duration(candidates[j][0]) < max_duration:
                    break
            playlist.append(candidates[j][0])
        start = end
        start_vec = end_vec
    playlist.append(end)
    return playlist

In [16]:
mp3tovec = pickle.load(open('../Pickles/mp3tovecs/mp3tovec.p', 'rb'))

In [17]:
playlist = join_the_dots([
    "H:\\Music\\Aretha Franklin\\I Never Loved a Man the Way I Love You\\01 Respect.mp3", # soul
    "H:\\Music\\James Brown\\The Godfather - The Very Best of James B\\02 I Got You (I Feel Good).m4a", # funk
    "H:\\Music\\Jurassic 5\\Jurassic 5 LP\\Lesson 6_ The Lecture.mp3", # hip-hop
    "H:\\Music\\Roni Size\\Breakbeat Era - Ultra Obscene\\Terrible Funk.mp3", # drum 'n' bass
    "H:\\Music\\Sven Väth\\In the Mix_ The Sound of the Sixteenth S\\14 Eclipse.m4a", # techno
], n=7, noise=0)

In [20]:
total_duration = 0
tracks = []
for track in playlist:
    tracks.append('-i')
    tracks.append(munge(track))
    total_duration += get_track_duration(track)
    print(f'{track}')
print(f'Total duration = {total_duration//60//60:.0f}:{total_duration//60%60:02.0f}:{total_duration%60:02.0f}s')

H:\Music\Aretha Franklin\I Never Loved a Man the Way I Love You\01 Respect.mp3
H:\Music\Dee Edwards\Gilles Peterson Digs America Vol.2\06 Why Can't There Be Love.mp3
H:\Music\Compilations\Northern Soul (The Soundtrack) [Extended\31 I Just Can_t Live My Life (Withou.m4a
H:\Music\Carnegie Mellon Jazz Band\Unknown Album\The First Thing I Do.mp3
H:\Music\Compilations\Mad Men (Music from the Series) Vol. 2\10 Pot Can't Talk About the Kettle.m4a
H:\Music\Tom Jones\Unknown Album\Ain't No Sunshine.mp3
H:\Music\Janis Joplin\The Essential Janis Joplin\14 Trust Me.m4a
H:\Music\James Brown\The Godfather - The Very Best of James B\02 I Got You (I Feel Good).m4a
H:\Music\Compilations\Mad Men (Music from the Series) Vol. 2\09 Let's Twist Again.m4a
H:\Music\Cut Chemist Feat. Hymnal\Paul's Second Compilation\06 What's The Altitude.mp3
H:\Music\Nicolas Repac\Swing swing\07 Swing Swing.mp3
H:\Music\Cut Chemist, DJ Shadow, Double Dee, Stei\The Ultimate Lessons\06 Lesson 6 - The Lecture (Original.m4a
H:\Mu

In [21]:
import subprocess as sp
pipe = sp.Popen(['ffmpeg',
                 '-y', # replace if exists
                '-i', '../static/meta_data.txt'] + # use this meta data
                tracks + # append playlist tracks
                 ['-filter_complex', f'loudnorm=I=-14,concat=n={len(playlist)}:v=0:a=1[out]', # normalize and concatenate
                 '-map', '[out]', # final output
                 'mix.mp3'], # output file
               stdin=sp.PIPE,stdout=sp.PIPE, stderr=sp.PIPE)

In [22]:
pipe.stderr.read()

b'ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers\n  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)\n  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid -

### Find a particular song, artist or album

In [19]:
for mp3 in mp3tovec:
    if mp3.lower().find('tribe') != -1:
        print('"'+ mp3.replace('\\', '\\\\') + '"')

"H:\\Music\\A Tribe Called Quest\\People's Instinctive Travels and the Pat\\06 Pubic Enemy.mp3"
"H:\\Music\\A Tribe Called Quest\\We got it from Here... Thank You 4 Your\\2-01 Mobius.mp3"
"H:\\Music\\A Tribe Called Quest\\People's Instinctive Travels and the Pat\\03 After Hours.mp3"
"H:\\Music\\A Tribe Called Quest\\We got it from Here... Thank You 4 Your\\1-03 Whateva Will Be.mp3"
"H:\\Music\\A Tribe Called Quest\\We got it from Here... Thank You 4 Your\\2-06 Conrad Tokyo.mp3"
"H:\\Music\\A Tribe Called Quest\\We got it from Here... Thank You 4 Your\\2-08 The Donald.mp3"
"H:\\Music\\A Tribe Called Quest\\We got it from Here... Thank You 4 Your\\1-01 The Space Program.mp3"
"H:\\Music\\A Tribe Called Quest\\We got it from Here... Thank You 4 Your\\2-07 Ego.mp3"
"H:\\Music\\A Tribe Called Quest\\We got it from Here... Thank You 4 Your\\1-04 Solid Wall of Sound.mp3"
"H:\\Music\\A Tribe Called Quest\\The Anthology\\02 Luck Of Lucien.mp3"
"H:\\Music\\A Tribe Called Quest\\The Low End Theory